# Extract the dataframe

In [1]:
# imports
import pandas as pd
import pyexasol

host = '192.168.56.101/8A3F422F336963EBB2E452E08A6B6E7060A554D85223032A44E9C60F0EFB8544:8563'  # Don't forget to change the fingerprint
user = 'sys'
password = 'exasol'

# Connect to Exasol
conn = pyexasol.connect(dsn=host, 
                        user=user, 
                        password=password, 
                        debug=False, 
                        protocol_version=pyexasol.PROTOCOL_V1)

In [9]:
q = '''
SELECT QUERYDIM.QUERY, TIMEDIM.[month], TIMEDIM.[day of the month], TIMEDIM.[hour], TIMEDIM.[minute], FACTS.ANONID, URLDIM.URL
FROM AOL_SCHEMA.QUERYDIM 
    LEFT JOIN AOL_SCHEMA.FACTS ON QUERYDIM.ID = FACTS.QUERYID
    LEFT JOIN AOL_SCHEMA.TIMEDIM ON FACTS.TIMEID = TIMEDIM.ID
    LEFT JOIN AOL_SCHEMA.URLDIM ON FACTS.URLID = URLDIM.ID
WHERE FACTS.CLICK = 1         
    AND URLDIM.URL IN (
        SELECT URLDIM.URL
        FROM AOL_SCHEMA.FACTS 
        JOIN AOL_SCHEMA.URLDIM ON AOL_SCHEMA.FACTS.URLID = AOL_SCHEMA.URLDIM.ID
        WHERE AOL_SCHEMA.FACTS.CLICK = 1
        GROUP BY URLDIM.URL
        ORDER BY COUNT(AOL_SCHEMA.FACTS.CLICK) DESC
        LIMIT 15
        )
    AND (QUERYDIM.QUERY IS NOT NULL
        AND TIMEDIM.[hour] IS NOT NULL
        AND TIMEDIM.[minute] IS NOT NULL
        AND TIMEDIM.[day of the year] IS NOT NULL)
'''

df = conn.export_to_pandas(q)
print("Finished SQL Query. Writing to CSV")
df.to_csv("./../Data/AOLQueries.csv")

Finished SQL Query. Writing to CSV
